In [ ]:
# Required imports
import pandas as pd
import openai
import json

# Load data from ta_new.json into a DataFrame
with open('[language].json', 'r', encoding='utf-8') as file:
    data = json.load(file)

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels
})

openai.api_key = '[api_key]'

def get_gpt_response(claim, language, model="gpt-4o", method="0shot"):
    system_prompt = f"You are an expert in multilingual understanding in {language} who gives to the point responses."
    user_prompt = f"'{claim}' Is this claim 'true', 'mostly true', 'partially true/misleading', ‘mostly false’, or ‘false’? You must always make sure your final response is prefixed with 'Final Answer:' followed by either ‘True’, ‘Mostly True’ ‘Partially True/Misleading’, ‘Mostly False’, or ‘False’."
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return response['choices'][0]['message']['content']

# Define the prompting method and language
language = "Portuguese"
method = "0shot"

# Collect model outputs and track results
outputs = []
correct = 0
wrong = 0
inconclusive = 0

for index, row in df.iterrows():
    claim = row['claim']
    label = row['label']
    output = get_gpt_response(claim, language, method=method)
    
    # Print the model's output
    print(f"Model Output: {output}")
    
    # Extract final answer from the output
    final_answer = None
    if "final answer: true" in output.lower():
        final_answer = "true"
    elif "final answer: false" in output.lower():
        final_answer = "false"
    
    # Determine correctness or inconclusiveness
    if final_answer is None:
        print("Inconclusive response")
        inconclusive += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            correct += 1
        else:
            print("Wrong response")
            wrong += 1
    
    # Save outputs
    outputs.append({
        'claim': claim,
        'label': label,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    })

# Save results to CSV
output_df = pd.DataFrame(outputs)
output_filename = f"{language}_gpt4o_{method}.csv"
output_df.to_csv(output_filename, index=False, encoding='utf-8')

# Summary of results
summary = {
    'correct': correct,
    'wrong': wrong,
    'inconclusive': inconclusive,
    'total': len(df)
}

summary_filename = f"{language}_gpt4o_{method}_summary.json"
with open(summary_filename, 'w', encoding='utf-8') as file:
    json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_filename} and {summary_filename}")
print(f"Summary: {summary}")
